<h3>Twitter API &amp; Tweepy</h3>
<p>Prima di poter iniziare ad utilizzare le API è necessario creare una applicazione associata all'accounta per sviluppatori di twitter.<br>
La creazione di un app può essere effettuata al seguente <a href="https://developer.twitter.com/en/apps">url</a>.
<p>Dopo aver creato l'applicazione è necessario generare un <b>Access Token</b> ed un <b>Access Token Secret</b>. E' inoltre necessario tenere nota della <b>API Key</b> e della <b>API Secret Key</b>.<br>
Nel mio caso i dati sono : <br>
<table>
    <tr>
        <th>Name</th>
        <th>Value</th>
    </tr>
    <tr>
        <td>API key</td>
        <td>XjEcjaRjminAol236APwFbDBL</td>
    </tr>
    <tr>
        <td>API secret key</td>
        <td>yHab5rIP810bZHeOD5stqREQMq4zSB6TPms3ezQMC1mJhlvryp</td>
    </tr>
    <tr>
        <td>Access Token</td>
        <td>1281151075723161601-qxLwXj6IoUrYjTwVp7bDK2tTTNooUR</td>
    </tr>
    <tr>
        <td>Access Token Secret</td>
        <td>t1kPqetMXMFrz6QRj2mzN1M2LZ5fzddlV5zVYxH3amYxN</td>
    </tr>
</table>
<p>La <b>API Key</b> e la <b>API Secret Key</b> sono le credenziali dell'applicazione necessarie per poter effettuare richieste alle API.<br>
<b>Access Token</b> e <b>Access Token Secret</b> sono invece valori necessari all'autenticazione degli utenti. Nel nostro caso l'utente coincide con lo sviluppatore ed è possibile generare automaticamente i valori delle chiavi. Per farlo è necessario andare nella sezione <b>Keys and Tokens</b> della pagina dell'applicazione.
<p>Queste informazioni sono sensibili e vanno protette al meglio per evitare che altri utenti possano effettuare richieste a nostro carico od ottenere accesso al nostro account.
<p>La documentazione relativa alla libreria Tweepy è disponibile al seguente <a href="http://docs.tweepy.org/en/latest/getting_started.html">link</a>.

<h5>Esempio</h5>
<p>Vediamo come ottenere i tweet più recenti associati all'account di Donald (Duck) Trump.
<p>Come prima cosa importiamo la libreria <code>tweepy</code> ed eventualmente altre librerie necessarie all'esecuzione del programma

In [1]:
import csv
import tweepy as tw
import pandas as pd

ModuleNotFoundError: No module named 'tweepy'

<p>Per aumentare la riusabilità del codice definiamo le funzionalità che ci interessano all'interno di una o più funzioni. In questo caso ho definito due funzioni : <code>process_status(tweet)</code> e <code>get_user_tweets(user, count, file_path)</code>.
<p>La funzione <code>process_status(tweet)</code> è una funzione di supporto per il parsing dei dati ritornati dalle API di twitter. Quest'ultimi contengono infatti svariate informazioni irrilevanti al caso di studio, se interessata è possibile visualizzare il contenuto della risposta semplicemente stampando a schermo il parametro <code>tweet</code> all'interno della funzione (basta che togli il commento e riesegui il codice!).

In [ ]:
def process_status(tweet):
    """
        Process a given tweet extrapolating the tweet's text, id, date and the user 
        that posted it.
        
        Params : 
        --------
            tweet : tweepy.Status -> the tweet to process.
            
        Return :
        --------
            date_create, user, tweet_id, text : str, tuple, str, str -> a tuple consisting of
            a string representing the date the tweet was posted, a tuple representing the
            user with his/her name, id, and screen name, a string representing the tweet 
            unique id and the tweet's text.
    """
    if hasattr(tweet, 'retweeted_status'):
        try:
            text = tweet.retweeted_status._json['full_text']
        except:
            text = tweet.retweeted_status._json['text']
    else:
        try:
            text = tweet._json['full_text']
        except AttributeError:
            text = tweet.text
            
    date_created = tweet.created_at
    user = (tweet.user.name, tweet.user.id, tweet.user.screen_name)
    tweet_id = tweet.id
    
    return date_created, user, tweet_id, text

<p>La seconda funzione è invece quella responsabile di effettuare la chiamata vera e propria. Essa accetta tre parametri : user, count e file_path. <code>user</code> è una stringa che rappresenta lo <code>'screen_name'</code> dell'utente ( nel caso di Trump è "potus"), <code>count</code> è un intero e viene utilizzato per limitare il numero di tweet da ritornare; <code>file_path</code> è invece una stringa che rappresenta il path del file in cui salvare i risultati.
<p>Le prime quattro righe sono semplici dichiarazioni di variabili contenenti le chiavi menzionate all'inizio del notebook. Le ho salvate come variabili per comodità ma attenta che è un pessimo approcio. Queste informazioni vanno infatti protette e lasciarle in chiaro nel codice sorgente è una brutta pratica!
<p>Successivamente viene effettuata l'autenticazione dell'applicazione e dell'utente per cui l'applicazione effettua le richieste. Questo processo è svolto definendo un'istanza della classe <code>OAuthHandler(key, secret)</code> dove <code>key</code> e <code>secret</code> rappresentano rispettivamente la API Key e la API Secret Key. Successivamente vengono inserite le credenziali utente tramite il metodo <code>set_access_token(token, secret)</code> della classe <code>OAuthHandler</code>.
<p>Viene poi definito un nuovo oggetto della classe <code>API</code> passando al costruttore l'oggetto di autenticazione definito precedentemente. Questo oggetto contiene dei wrapper per effettuare le chiamate http agli end-point delle API di Twitter rendendo più semplice e veloce lo sviluppo. Uno dei wrapper esposti dalla classe <code>API</code> è <code>user_timeline()</code> che permette, specificato il nome o l'id di un utente di ottenere una lista dei suoi tweets. 
<p>Questo wrapper viene utilizzato all'interno della classe <code>Cursor</code> che semplicemente facilita l'iterazione sui tweets come si può notare dalla presenza del ciclo <code>for</code>.<br>
All'interno del ciclo la variabile <code>status</code> rappresenta i dati grezzi come ritornati dalle API, questa variabile viene poi passata alla funzione di supporto precedentemente definita per estrapolare i dati d'interesse. I valori ritornati dalla funzione di supporto sono spacchettati e successivamente scritti su file. Per quanto riguarda la gestione dei file in Python ti rimando alla <a href="https://docs.python.org/3/tutorial/inputoutput.html">documentazione</a>.

In [ ]:
def get_user_tweets(user, count, file_path):
    consumer_key = "XjEcjaRjminAol236APwFbDBL"
    consumer_secret = "yHab5rIP810bZHeOD5stqREQMq4zSB6TPms3ezQMC1mJhlvryp"
    access_token = "1281151075723161601-qxLwXj6IoUrYjTwVp7bDK2tTTNooUR"
    access_token_secret = "t1kPqetMXMFrz6QRj2mzN1M2LZ5fzddlV5zVYxH3amYxN"

    auth = tw.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    api = tw.API(auth)
    
    f = open(file_path, "w")
    csv_writer = csv.writer(f)
    
    for status in tw.Cursor(api.user_timeline, id=user, tweet_mode='extended').items(count):
        date, user, _id, text = process_status(status)
        csv_writer.writerow([date, _id, text, user[1], user[2]])
        
    f.close()

In [ ]:
get_user_tweets("potus", 5, "./ciao.csv")

In [ ]:
df = pd.read_csv(
    "ciao.csv",
    sep=",",
    header=0,
    names=['date', 'tweet_id', 'tweet', 'user_screen_name', 'user_id'],
    index_col=['date'],
    parse_dates=['date']
)
df.head()

In [ ]:
def get_tweets_by_query(query, count, file_path):
    consumer_key = "XjEcjaRjminAol236APwFbDBL"
    consumer_secret = "yHab5rIP810bZHeOD5stqREQMq4zSB6TPms3ezQMC1mJhlvryp"
    access_token = "1281151075723161601-qxLwXj6IoUrYjTwVp7bDK2tTTNooUR"
    access_token_secret = "t1kPqetMXMFrz6QRj2mzN1M2LZ5fzddlV5zVYxH3amYxN"

    auth = tw.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    api = tw.API(auth)
    
    f = open(file_path, "w")
    csv_writer = csv.writer(f)
    
    for status in tw.Cursor(api.search, q=query, tweet_mode='extended').items(count):
        print(status)
        
        
get_tweets_by_query("@potus", 5, "test.csv")